In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import csr_matrix

import time

import memory_profiler
%load_ext memory_profiler

from pathlib import Path

In [2]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [4]:
%load_ext autoreload
%autoreload 2

from pubmed_landscape_src.metrics import knn_covid

In [3]:
variables_path = Path("../results/variables")
figures_path = Path("../results/figures")
berenslab_data_path = Path("/gpfs01/berens/data/data/pubmed_processed")

# Import

In [4]:
tfidf_features_updated_reparsed = sp.sparse.load_npz(variables_path / "tfidf_features_updated_reparsed.npz")

tcmalloc: large alloc 18038185984 bytes == 0x4d64000 @ 
tcmalloc: large alloc 18038185984 bytes == 0x43930a000 @ 


In [5]:
tfidf_features_updated_reparsed.shape

(20687150, 4679130)

In [13]:
%%time
%%memit
embeddings_reparsed_all=np.load(berenslab_data_path / 'embeddings/embeddings_reparsed_all.npy')

tcmalloc: large alloc 127101853696 bytes == 0x21aae88000 @ 


peak memory: 156318.00 MiB, increment: 18199.42 MiB
CPU times: user 184 ms, sys: 1min 12s, total: 1min 12s
Wall time: 1min 44s


In [3]:
tsne_tfidf_reparsed=np.load(variables_path / "tsne_tfidf_reparsed.npy")

In [4]:
tsne_bert_reparsed=np.load(variables_path / "bert/tsne_reparsed.npy")

In [15]:
tfidf_features_updated_reparsed.shape

(20687150, 4679130)

# Isolatedness (kNN covid)

## Keywords: Covid-19

In [ ]:
# import clean_df_updated
clean_df_updated_reparsed_filtered = pd.read_pickle(variables_path / "clean_df_updated_reparsed_filtered")

In [5]:
titles=clean_df_updated_reparsed_filtered['AbstractText']
sub1='covid-19'
sub2='COVID-19'
sub3='Covid-19'
sub4='CoViD-19'
sub5='2019-nCoV'
sub6='SARS-CoV-2'
sub7='coronavirus disease 2019'
sub8='Coronavirus disease 2019'

indexes1= titles.str.find(sub1)
indexes2= titles.str.find(sub2)
indexes3= titles.str.find(sub3)
indexes4= titles.str.find(sub4)
indexes5= titles.str.find(sub5)
indexes6= titles.str.find(sub6)
indexes7= titles.str.find(sub7)
indexes8= titles.str.find(sub8)

mask_covid_2 = (indexes1!=-1) | (indexes2!=-1) | (indexes3!=-1) | (indexes4!=-1) | (indexes5!=-1) | (indexes6!=-1) | (indexes7!=-1) | (indexes8!=-1)

#information
non_1_1=indexes1[indexes1!=-1]
print('Number of titles with word 1: ',len(non_1_1))
non_1_2=indexes2[indexes2!=-1]
print('Number of titles with word 2: ',len(non_1_2))
non_1_3=indexes2[indexes3!=-1]
print('Number of titles with word 3: ',len(non_1_3))
non_1_4=indexes2[indexes4!=-1]
print('Number of titles with word 4: ',len(non_1_4))
non_1_5=indexes2[indexes5!=-1]
print('Number of titles with word 5: ',len(non_1_5))
non_1_6=indexes2[indexes6!=-1]
print('Number of titles with word 6: ',len(non_1_6))
non_1_7=indexes2[indexes7!=-1]
print('Number of titles with word 7: ',len(non_1_7))
non_1_8=indexes2[indexes8!=-1]
print('Number of titles with word 8: ',len(non_1_8))

nsc_titles_word=titles[(indexes1!=-1) | (indexes2!=-1) | (indexes3!=-1) | (indexes4!=-1) | (indexes5!=-1) | (indexes6!=-1) | (indexes7!=-1) | (indexes8!=-1)]
print('Number of titles with word: ',len(nsc_titles_word))

Number of titles with word 1:  521
Number of titles with word 2:  115703
Number of titles with word 3:  4474
Number of titles with word 4:  66
Number of titles with word 5:  948
Number of titles with word 6:  45386
Number of titles with word 7:  22193
Number of titles with word 8:  4761
Number of titles with word:  132802


In [11]:
np.save(variables_path / 'mask_covid_2', mask_covid_2)

In [7]:
mask_covid_2 = np.load(variables_path / 'mask_covid_2.npy')

### TF-IDF

In [ ]:
%%time
knn_covid_tfidf_5000 = knn_covid(tfidf_features_updated_reparsed, mask_covid_2, k=10, subset_size=5000)

tcmalloc: large alloc 18038185984 bytes == 0x8db65a000 @ 
tcmalloc: large alloc 18038185984 bytes == 0xd0f900000 @ 


In [ ]:
np.save(variables_path / 'knn_covid_tfidf_5000', knn_covid_tfidf_5000)

In [16]:
print(knn_covid_tfidf_5000)

0.7617200000000001


### BERT

In [ ]:
%%capture cap
%%time
%%memit
knn_covid_bert_5000 = knn_covid(embeddings_reparsed_all, mask_covid_2, k=10, subset_size=5000)

In [ ]:
with open( variables_path / 'verbose_knn_covid_bert_5000.txt', 'w') as f:
    f.write(cap.stdout)

In [70]:
print(knn_covid_bert_5000)

0.8056599999999999


In [ ]:
np.save(variables_path / 'knn_covid_bert_5000', knn_covid_bert_5000)

### t-SNE TF-IDF

In [10]:
%%time
knn_covid_tnse_tfidf = knn_covid(tsne_tfidf_reparsed, mask_covid_2, k=10, subset_size=10000)

CPU times: user 31.6 s, sys: 638 ms, total: 32.3 s
Wall time: 31.1 s


In [11]:
np.save(variables_path / 'knn_covid_tnse_tfidf', knn_covid_tnse_tfidf)

In [12]:
print(knn_covid_tnse_tfidf)

0.8452299999999999


### t-SNE BERT

In [13]:
%%time
knn_covid_tsne_bert = knn_covid(tsne_bert_reparsed, mask_covid_2, k=10, subset_size=10000)

CPU times: user 29 s, sys: 501 ms, total: 29.5 s
Wall time: 27.9 s


In [14]:
print(knn_covid_tsne_bert)

0.65445


In [15]:
np.save(variables_path / 'knn_covid_tsne_bert', knn_covid_tsne_bert)

## Keyword: HIV

In [4]:
# import clean_df_updated
clean_df_updated_reparsed_filtered = pd.read_pickle(variables_path / "clean_df_updated_reparsed_filtered")

tcmalloc: large alloc 1073741824 bytes == 0x8c1ad4000 @ 
tcmalloc: large alloc 1233903616 bytes == 0x96ba6c000 @ 


In [5]:
#Neuroscience colors

#Buscar 
titles=clean_df_updated_reparsed_filtered['AbstractText']
sub1='HIV'
sub2=' hiv'
sub3='AIDS'
sub4=' aids'

indexes1= titles.str.find(sub1)
indexes2= titles.str.find(sub2)
indexes3= titles.str.find(sub3)
indexes4= titles.str.find(sub4)

mask_hiv = (indexes1!=-1) | (indexes2!=-1) | (indexes3!=-1) | (indexes4!=-1)

#information
non_1_1=indexes1[indexes1!=-1]
print('Number of titles with word 1: ',len(non_1_1))
non_1_2=indexes2[indexes2!=-1]
print('Number of titles with word 2: ',len(non_1_2))
non_1_3=indexes2[indexes3!=-1]
print('Number of titles with word 3: ',len(non_1_3))
non_1_4=indexes2[indexes4!=-1]
print('Number of titles with word 4: ',len(non_1_4))

nsc_titles_word=titles[(indexes1!=-1) | (indexes2!=-1) | (indexes3!=-1) | (indexes4!=-1)]
print('Number of titles with word: ',len(nsc_titles_word))

Number of titles with word 1:  260386
Number of titles with word 2:  1958
Number of titles with word 3:  75537
Number of titles with word 4:  25189
Number of titles with word:  308077


In [6]:
np.save(variables_path / 'mask_hiv', mask_hiv)

### TF-IDF

In [ ]:
%%capture cap
%%time
knn_hiv_tfidf_5000 = knn_covid(tfidf_features_updated_reparsed, mask_hiv, k=10, subset_size=5000)

In [ ]:
with open( variables_path / 'verbose_knn_hiv_tfidf_5000.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
np.save(variables_path / 'knn_hiv_tfidf_5000', knn_hiv_tfidf_5000)

In [71]:
print(knn_hiv_tfidf_5000)

0.62312


### BERT

In [ ]:
%%capture cap
%%time
knn_hiv_bert_5000 = knn_covid(embeddings_reparsed_all, mask_hiv, k=10, subset_size=5000)

In [ ]:
with open( variables_path / 'verbose_knn_hiv_bert_5000.txt', 'w') as f:
    f.write(cap.stdout)

In [72]:
print(knn_hiv_bert_5000)

0.6386000000000001


In [ ]:
np.save(variables_path / 'knn_hiv_bert_5000', knn_hiv_bert_5000)

## Keyword: influenza

In [ ]:
# import clean_df_updated
clean_df_updated_reparsed_filtered = pd.read_pickle(variables_path / "clean_df_updated_reparsed_filtered")

In [10]:
#Neuroscience colors

#Buscar neuroscience
titles=clean_df_updated_reparsed_filtered['AbstractText']
word='influenza'
sub1=' '+word
sub2=word.capitalize()

indexes1= titles.str.find(sub1)
indexes2= titles.str.find(sub2)

mask_influenza = (indexes1!=-1) | (indexes2!=-1) 

#information
non_1_1=indexes1[indexes1!=-1]
print('Number of titles with word 1: ',len(non_1_1))
non_1_2=indexes2[indexes2!=-1]
print('Number of titles with word 2: ',len(non_1_2))

nsc_titles_word=titles[(indexes1!=-1) | (indexes2!=-1)]
#print('Number of titles with word small and capital: ',len(non_1_1)+len(non_1_2)-len(nsc_titles_word))
print('Number of titles with word: ',len(nsc_titles_word))


Number of titles with word 1:  88012
Number of titles with word 2:  13001
Number of titles with word:  90575


In [11]:
np.save(variables_path / 'mask_influenza', mask_influenza)

### TF-IDF

In [ ]:
%%time
knn_influenza_tfidf_5000 = knn_covid(tfidf_features_updated_reparsed, mask_influenza, k=10, subset_size=5000)

In [ ]:
np.save(variables_path / 'knn_influenza_tfidf_5000', knn_influenza_tfidf_5000)

In [ ]:
#knn_influenza_tfidf_5000 = np.load(knn_influenza_tfidf_5000 / 'knn_influenza_tfidf_5000.npy')

In [73]:
print(knn_influenza_tfidf_5000)

0.64142


### BERT

In [ ]:
knn_influenza_bert_5000 = knn_covid(embeddings_reparsed_all, mask_influenza, k=10, subset_size=5000)

In [74]:
print(knn_influenza_bert_5000)

0.57938


In [ ]:
np.save(variables_path / 'knn_influenza_bert_5000', knn_influenza_bert_5000)

## Label: virology

In [24]:
# load
# legend colors
pickle_in = open(variables_path / "colors_new_labels_with_more_legend.pkl","rb")
colors_new_labels_with_more_legend = pickle.load(pickle_in)

colors_new_labels_with_more = np.load(variables_path / "colors_new_labels_with_more.npy", allow_pickle = True)

In [25]:
colors_new_labels_with_more_legend['virology']

'#6F0062'

In [26]:
mask_virology = colors_new_labels_with_more == colors_new_labels_with_more_legend['virology']

In [27]:
np.sum(mask_virology)

112807

### TF-IDF

In [ ]:
%%time
knn_virology_tfidf_5000 = knn_covid(tfidf_features_updated_reparsed, mask_virology, k=10, subset_size=5000)

In [ ]:
np.save(variables_path / 'knn_virology_tfidf_5000', knn_virology_tfidf_5000)

In [75]:
print(knn_virology_tfidf_5000)

0.39146000000000003


### BERT

In [ ]:
%%time
knn_virology_bert_5000 = knn_covid(embeddings_reparsed_all, mask_virology, k=10, subset_size=5000)

In [76]:
print(knn_virology_bert_5000)

0.47737999999999997


In [ ]:
np.save(variables_path / 'knn_virology_bert_5000', knn_virology_bert_5000)

## Label: ophtalmology

In [3]:
# load
# legend colors
pickle_in = open(variables_path / "colors_new_labels_with_more_legend.pkl","rb")
colors_new_labels_with_more_legend = pickle.load(pickle_in)

colors_new_labels_with_more = np.load(variables_path / "colors_new_labels_with_more.npy", allow_pickle = True)

In [28]:
colors_new_labels_with_more_legend['ophthalmology']

'#A4E804'

In [29]:
mask_ophthalmology = colors_new_labels_with_more == colors_new_labels_with_more_legend['ophthalmology']

In [30]:
np.sum(mask_ophthalmology)

144411

### TF-IDF

In [ ]:
%%time
knn_ophthalmology_tfidf_5000 = knn_covid(tfidf_features_updated_reparsed, mask_ophthalmology, k=10, subset_size=5000)

In [ ]:
np.save(variables_path / 'knn_ophthalmology_tfidf_5000', knn_ophthalmology_tfidf_5000)

In [77]:
print(knn_ophthalmology_tfidf_5000)

0.43632


### BERT

In [ ]:
%%time
knn_ophthalmology_bert_5000 = knn_covid(embeddings_reparsed_all, mask_ophthalmology, k=10, subset_size=5000)

In [78]:
print(knn_ophthalmology_bert_5000)

0.47653999999999996


In [ ]:
np.save(variables_path / 'knn_ophthalmology_bert_5000', knn_ophthalmology_bert_5000)

## Keyword: meta-analysis

In [ ]:
# import clean_df_updated
clean_df_updated_reparsed_filtered = pd.read_pickle(variables_path / "clean_df_updated_reparsed_filtered")

In [7]:
#Neuroscience colors

#Buscar neuroscience
titles=clean_df_updated_reparsed_filtered['AbstractText']
word='meta-analysis'
sub1=' '+word
sub2=word.capitalize()

indexes1= titles.str.find(sub1)
indexes2= titles.str.find(sub2)

mask_meta_analysis = (indexes1!=-1) | (indexes2!=-1) 

#information
non_1_1=indexes1[indexes1!=-1]
print('Number of titles with word 1: ',len(non_1_1))
non_1_2=indexes2[indexes2!=-1]
print('Number of titles with word 2: ',len(non_1_2))

nsc_titles_word=titles[(indexes1!=-1) | (indexes2!=-1)]
#print('Number of titles with word small and capital: ',len(non_1_1)+len(non_1_2)-len(nsc_titles_word))
print('Number of titles with word: ',len(nsc_titles_word))


Number of titles with word 1:  138378
Number of titles with word 2:  16100
Number of titles with word:  145358


In [8]:
np.save(variables_path_path / 'mask_meta_analysis', mask_meta_analysis)

### TF-IDF

In [ ]:
%%time
knn_meta_analysis_tfidf_5000 = knn_covid(tfidf_features_updated_reparsed, mask_meta_analysis, k=10, subset_size=5000)

In [ ]:
np.save(variables_path / 'knn_meta_analysis_tfidf_5000', knn_meta_analysis_tfidf_5000)

In [79]:
print(knn_meta_analysis_tfidf_5000)

0.38472


In [4]:
#knn_meta_analysis_tfidf_5000 = np.load(variables_path / 'knn_meta_analysis_tfidf_5000.npy')

### BERT

In [ ]:
knn_meta_analysis_bert_5000 = knn_covid(embeddings_reparsed_all, mask_meta_analysis, k=10, subset_size=5000)

In [80]:
print(knn_meta_analysis_bert_5000)

0.52574


In [ ]:
np.save(variables_path / 'knn_meta_analysis_bert_5000', knn_meta_analysis_bert_5000)

## Keyword: new species

In [ ]:
# import clean_df_updated
clean_df_updated_reparsed_filtered = pd.read_pickle(variables_path / "clean_df_updated_reparsed_filtered")

In [82]:
#Neuroscience colors

#Buscar neuroscience
titles=clean_df_updated_reparsed_filtered['AbstractText']
word='new species'
sub1=' '+word
sub2=word.capitalize()

indexes1= titles.str.find(sub1)
indexes2= titles.str.find(sub2)

mask_new_species = (indexes1!=-1) | (indexes2!=-1) 

#information
non_1_1=indexes1[indexes1!=-1]
print('Number of titles with word 1: ',len(non_1_1))
non_1_2=indexes2[indexes2!=-1]
print('Number of titles with word 2: ',len(non_1_2))

nsc_titles_word=titles[(indexes1!=-1) | (indexes2!=-1)]
#print('Number of titles with word small and capital: ',len(non_1_1)+len(non_1_2)-len(nsc_titles_word))
print('Number of titles with word: ',len(nsc_titles_word))


Number of titles with word 1:  27198
Number of titles with word 2:  233
Number of titles with word:  27354


### TF-IDF

In [ ]:
%%time
knn_new_species_tfidf_5000 = knn_covid(tfidf_features_updated_reparsed, mask_new_species, k=10, subset_size=5000)

tcmalloc: large alloc 18038185984 bytes == 0xd0f900000 @ 
tcmalloc: large alloc 18038185984 bytes == 0x3f5e016000 @ 


In [ ]:
np.save(variables_path / 'knn_new_species_tfidf_5000', knn_new_species_tfidf_5000)

In [90]:
print(knn_new_species_tfidf_5000)

0.40484


### BERT

In [ ]:
knn_new_species_bert_5000 = knn_covid(embeddings_reparsed_all, mask_new_species, k=10, subset_size=5000)

In [91]:
print(knn_new_species_bert_5000)

0.52732


In [ ]:
np.save(variables_path / 'knn_new_species_bert_5000', knn_new_species_bert_5000)